장르 종합 주간차트 (2025년 1 ~ 11월) 기준 <hr>

가수 유형 분석

In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import os

In [ ]:
# ============================================================================
# 1. Selenium 드라이버 초기화 및 멜론 웹사이트 접속
# ============================================================================

# Chrome 드라이버 생성
driver = webdriver.Chrome()

# 멜론 메인 페이지 접속
driver.get('https://www.melon.com')

# WebDriverWait 객체 생성 (최대 10초 대기)
wait = WebDriverWait(driver, 10)

# "멜론차트" 링크를 클릭할 때까지 대기 후 클릭
wait.until(EC.element_to_be_clickable((By.LINK_TEXT, "멜론차트"))).click()
time.sleep(2)  # 페이지 로드 대기

# ============================================================================
# 2. 월별 크롤링 반복 (1월 ~ 11월)
# ============================================================================

for month in range(1, 12):  # month = 1, 2, 3, ..., 11
    print(f"\n{'='*70}")
    print(f"{'='*25} {month}월 크롤링 시작 {'='*25}")
    print(f"{'='*70}\n")
    
    # 월별 폴더 생성 (예: melon_2025_01month)
    folder_name = f"melon_2025_{month:02d}month"
    os.makedirs(folder_name, exist_ok=True)  # 폴더가 없으면 생성, 있으면 통과
    print(f"📁 폴더: {folder_name}")
    
    # ========================================================================
    # 3. 차트파인더 메뉴 열기
    # ========================================================================
    
    # 차트파인더 버튼 클릭
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#gnb_menu > ul:nth-child(1) > li.nth1.on > div > div > button > span"))).click()
    time.sleep(1)
    
    # ========================================================================
    # 4. 차트 옵션 설정 (주간차트, 2020년대, 2025년, 해당 월)
    # ========================================================================
    def click_chart(wait, css_selector):
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector))).click()
        time.sleep(1)

    
    wait = WebDriverWait(driver, 10)    # 클릭 가능한 상태가 될 때까지 최대 10초 대기

    click_chart(wait, "a[data-value='WE']")             # 주간차트 선택
    click_chart(wait, "label[for='decade_1']")          # 2020년대 선택
    click_chart(wait, "label[for='year_1']")            # 2025년 선택
    click_chart(wait, f"label[for='month_{month}']")    # 월 선택    
    
    # ========================================================================
    # 5. 해당 월의 주차 개수 확인
    # ========================================================================
    
    # week_1 ~ week_5 중 존재하는 요소 개수를 센다
    # (예: 1월에 4주가 있으면 week_count = 4)
    week_count = sum(1 for w in range(1, 6) if driver.find_elements(By.ID, f"week_{w}"))
    print(f"📅 {month}월에는 {week_count}개의 주차가 있습니다.\n")
    
    # ========================================================================
    # 6. 각 주차별 크롤링 반복
    # ========================================================================
    
    for week in range(1, week_count + 1):  # week = 1, 2, 3, ..., week_count
        print(f"{'─'*70}")
        print(f"  {month}월 {week}주차 크롤링 시작")
        print(f"{'─'*70}")
        
        # ====================================================================
        # 7. 주차 및 장르 선택 후 검색
        # ====================================================================
        
        # 해당 주차 선택
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"label[for='week_{week}']"))).click()
        time.sleep(1)
        
        # 장르: 종합 선택
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='gnr_1']"))).click()
        time.sleep(1)
        
        # 검색 버튼 클릭
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn_b26[type='submit']"))).click()
        time.sleep(3)  # 검색 결과 로드 대기
        
        # ====================================================================
        # 8. HTML 파싱 및 곡 정보 추출
        # ====================================================================
        
        # 현재 페이지의 HTML 파싱
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        data = []  # 크롤링한 곡 정보를 저장할 리스트
        
        print(f"1-100위 크롤링 중...")
        
        # 테이블의 tbody 아래 모든 tr(행)을 선택하고 처음 100개만 추출
        songs = soup.select('table > tbody > tr')[:100]
        
        # ====================================================================
        # 9. 각 곡의 정보 추출 (순위, 제목, 가수, 장르, 소속사)
        # ====================================================================
        
        for idx, song in enumerate(songs, 1):  # idx = 1, 2, 3, ..., 100
            # 순위 추출 (예: "1")
            rank = song.select_one('.rank').text.strip()
            
            # 노래 제목 추출
            title = song.select_one('.ellipsis.rank01 a').text.strip()
            
            # 가수명 추출
            artist = song.select_one('.ellipsis.rank02 a').text.strip()
            
            # 초기값 설정 (아티스트 정보는 별도 페이지에서 크롤링)
            artist_type = "정보없음"
            company = "정보없음"
            
            # ================================================================
            # 10. 아티스트 정보 크롤링 (새 창에서)
            # ================================================================
            
            # 아티스트 링크 가져오기
            artist_link = song.select_one('.ellipsis.rank02 a')
            artist_href = artist_link.get('href')  # href 속성 추출
            
            # href가 있고 'goArtistDetail'을 포함하면 진행
            if artist_href and 'goArtistDetail' in artist_href:
                # 예: "javascript:goArtistDetail('123456')" 에서 artist_id 추출
                artist_id = artist_href.split("'")[1]
                
                # 아티스트 페이지 새 창에서 열기
                driver.execute_script(f"window.open('https://www.melon.com/artist/timeline.htm?artistId={artist_id}', '_blank');")
                
                # 새 창으로 전환
                driver.switch_to.window(driver.window_handles[-1])
                
                # 아티스트 정보 로드 대기
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "dl.atist_info")))
                time.sleep(1)
                
                # 아티스트 페이지 HTML 파싱
                soup_artist = BeautifulSoup(driver.page_source, 'html.parser')
                atist_info = soup_artist.select_one('dl.atist_info')  # 아티스트 정보 영역
                
                # ============================================================
                # 11. 아티스트 정보 추출 (활동유형, 소속사)
                # ============================================================
                if atist_info:
                    # dt(제목)와 dd(내용)을 모두 추출
                    elements = atist_info.find_all(['dt', 'dd'])
                    current_dt = None  # 초기화
                    
                    for elem in elements:
                        if elem.name == 'dt':
                            # dt 태그면 현재 제목 저장 (예: "활동유형", "소속사")
                            current_dt = elem.text.strip()
                        elif elem.name == 'dd' and current_dt:
                            # dd 태그면 해당 제목의 값 추출
                            dd_text = elem.text.strip()
                            
                            # 활동유형 추출 (그룹, 솔로, 듀엣 등)
                            if current_dt == '활동유형':
                                for type_name in ['그룹', '솔로']:
                                    if type_name in dd_text:
                                        artist_type = dd_text
                                        break
                            # 소속사 추출
                            elif current_dt == '소속사':
                                company = dd_text
                
                # 아티스트 페이지 창 닫기
                driver.close()
                
                # 원래 멜론차트 창으로 전환
                driver.switch_to.window(driver.window_handles[0])
            
            # ================================================================
            # 12. 수집한 정보를 리스트에 추가
            # ================================================================
            
            data.append({
                "순위": rank, 
                "노래제목": title, 
                "그룹이름": artist, 
                "가수유형": artist_type, 
                "소속사": company
            })
            
            # 진행상황 출력
            print(f"    [{idx}/100] {rank}위 - {title[:20]:20s} | {artist[:15]:15s} | {artist_type:8s} | {company[:20]:20s}")
        
        # ====================================================================
        # 13. 크롤링한 데이터를 CSV로 저장
        # ====================================================================
        
        # 리스트를 DataFrame으로 변환
        df = pd.DataFrame(data)
        
        # 인덱스를 1부터 시작하도록 설정
        df.index = range(1, len(df) + 1)
        
        # CSV 파일로 저장 (예: melon_2025_01month/week_1.csv)
        df.to_csv(f"{folder_name}/week_{week}.csv", index=True, encoding='utf-8-sig')
        print(f"\n  저장 완료: {folder_name}/week_{week}.csv\n")
        
        # ====================================================================
        # 14. 다음 주차 크롤링을 위해 차트파인더 재설정
        # ====================================================================
        
        # 현재 주차가 마지막이 아니면 다음 주차를 위해 준비
        if week < week_count:
            # 차트 페이지로 이동
            driver.get('https://www.melon.com/chart/index.htm')
            time.sleep(2)
            
            # 차트파인더 열기
            wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#gnb_menu > ul:nth-child(1) > li.nth1.on > div > div > button > span"))).click()
            time.sleep(1)
            
            # 주간차트 선택
            wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[data-value='WE']"))).click()
            time.sleep(1)
            
            # 2020년대, 2025년, 해당 월 재선택
            wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='decade_1']"))).click()
            time.sleep(1)
            wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='year_1']"))).click()
            time.sleep(1)
            wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"label[for='month_{month}']"))).click()
            time.sleep(1)
    
    # ========================================================================
    # 15. 다음 월 크롤링을 위해 차트 페이지로 이동
    # ========================================================================
    
    # 현재 월이 11월이 아니면 다음 월을 위해 준비
    if month < 11:
        # 차트 메인 페이지로 이동
        driver.get('https://www.melon.com/chart/index.htm')
        time.sleep(2)

# ============================================================================
# 16. 크롤링 완료 및 드라이버 종료
# ============================================================================

# 드라이버 종료 (모든 창 닫기)
driver.quit()

# 완료 메시지 출력
print("\n" + "="*70)
print(f"{'전체 크롤링 완료!':^70}")
print("="*70)


========================= 1월 크롤링 시작 =========================

📁 폴더: melon_2025_01month


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=142.0.7444.176); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff7fe46a235
	0x7ff7fe1c2630
	0x7ff7fdf516dd
	0x7ff7fdf3d205
	0x7ff7fdf62a8a
	0x7ff7fdfda476
	0x7ff7fdffb3d2
	0x7ff7fdf9b0ed
	0x7ff7fdf9bf63
	0x7ff7fe495d60
	0x7ff7fe48fe8a
	0x7ff7fe4b1005
	0x7ff7fe1dd71e
	0x7ff7fe1e4e1f
	0x7ff7fe1cb7c4
	0x7ff7fe1cb97f
	0x7ff7fe1b18e8
	0x7ffdcc52e8d7
	0x7ffdccfec53c
